# Red Hat Business Value

We will use the dataset provided for the “Red Hat Business Value” competition. This competition was hosted on Kaggle a few years back, and the dataset is a really good business use case for our study. The archived competition is available at www.kaggle.com/c/predicting-red-hat-business-value.

## 1) Problem Statement

It highlights the problem that deals with predicting high-value customers for their business based on the operational interaction data and thereby helping the company effectively prioritize resources to generate more business and serve its customers better.

* The organization is an American multinational software company that provides open source software products to the enterprise community. 
* Their primary product is Red Hat Enterprise Linux, the most popular distribution of Linux OS, used by various large enterprises. 
* In its services, it helps organizations align their IT strategies by providing enterprise-grade solutions through an open business model and an affordable, predictable subscription model. 
* These subscriptions from large enterprise customers create a substantial part of their revenue, and therefore it is of paramount importance for them to understand their valuable customers and serve them better by prioritizing resources and strategies to drive improved business value.

Red Hat has been in existence for over 25 years. In the long stint of business, they have accumulated and captured a vast amount of data from customer interactions and their descriptive attributes. This rich source of data could be a gold mine of patterns that can help in identifying a potential customer by studying the vast and complex historical patterns in the interaction data.

With the ever-growing popularity and prowess of DL, we can develop a DNN that can learn from historic customer attributes and operational interaction data to understand the deep patterns and predict whether a new customer will potentially be a high-value customer for various business services.

## 2) Exploring the Data

In [219]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [220]:
#Import the 2 datasets provided in the Zip Folder
df = pd.read_csv("act_train.csv")
people = pd.read_csv("people.csv")

In [221]:
#Explore the shape of the datasets
print("Shape of Activities DF:",df.shape)
print("Shape of People DF:",people.shape)

Shape of Activities DF: (2197291, 15)
Shape of People DF: (189118, 41)


In [222]:
#Explore the contents of the first dataset
df.head()

,people_id,activity_id,date,activity_category,char_1,char_2,char_3,char_4,char_5,char_6,char_7,char_8,char_9,char_10,outcome
0,ppl_100,act2_1734928,2023-08-26,type 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 76,0
1,ppl_100,act2_2434093,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
2,ppl_100,act2_3404049,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
3,ppl_100,act2_3651215,2023-08-04,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
4,ppl_100,act2_4109017,2023-08-26,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0


Exploring the contents of the training dataset, we can see that it mostly has customer interaction data but is completely anonymized.

In [223]:
#Calculating the % of Null values in each column for activity data
df.isnull().sum()/df.shape[0]

people_id            0.000000
activity_id          0.000000
date                 0.000000
activity_category    0.000000
char_1               0.928268
char_2               0.928268
char_3               0.928268
char_4               0.928268
char_5               0.928268
char_6               0.928268
char_7               0.928268
char_8               0.928268
char_9               0.928268
char_10              0.071732
outcome              0.000000
dtype: float64

#### Around nine features have more than 90% null values. We can’t do much to fix these features. [char_1 to char_9]

In [224]:
#missing values/null values
df.isnull().sum().sum()

18514699

In [225]:
#Explore the contents of People dataset
people.head()

,people_id,char_1,group_1,char_2,date,char_3,char_4,char_5,char_6,char_7,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,ppl_100,type 2,group 17304,type 2,2021-06-29,type 5,type 5,type 5,type 3,type 11,...,False,True,True,False,False,True,True,True,False,36
1,ppl_100002,type 2,group 8688,type 3,2021-01-06,type 28,type 9,type 5,type 3,type 11,...,False,True,True,True,True,True,True,True,False,76
2,ppl_100003,type 2,group 33592,type 3,2022-06-10,type 4,type 8,type 5,type 2,type 5,...,False,False,True,True,True,True,False,True,True,99
3,ppl_100004,type 2,group 22593,type 3,2022-07-20,type 40,type 25,type 9,type 4,type 16,...,True,True,True,True,True,True,True,True,True,76
4,ppl_100006,type 2,group 6534,type 3,2022-07-27,type 40,type 25,type 9,type 3,type 8,...,False,False,True,False,False,False,True,True,False,84


In [226]:
#Calculate the % of null values in for the entire dataset
people.isnull().sum().sum()

0

#### And we see that none of the columns in the customer dataset has missing values. But before we do that, we need to take care of a few things as below:

* To create a consolidated dataset, we need to join the activity and customer data on the people_id key. 
* We need to drop the columns in the activity data that have 90% missing values, as they cannot be fixed. [char_1 to char_9]
* Secondly, the “date” and “char_10” columns are present in both datasets. 
* In order to avoid a nameclash, let us rename the “date” column in the activity dataset to “activity_date” and “char_10” in the activity data as “activity_type.” 
* Next, we also need to fix the missing values in the “activity_type” column. 
* Once these two tasks are accomplished, we will join the two datasets and explore the consolidated data.

In [227]:
#Create the list of columns to drop from activity data
columns_to_remove = ["char_"+str(x) for x in np.arange(1,10)]
print("Columns to remove:",columns_to_remove)

Columns to remove: ['char_1', 'char_2', 'char_3', 'char_4', 'char_5', 'char_6', 'char_7', 'char_8', 'char_9']


In [228]:
#Remove the columns from the activity data
df = df[list(set(df.columns) - set(columns_to_remove))]

In [229]:
df.head()

,outcome,char_10,people_id,date,activity_category,activity_id
0,0,type 76,ppl_100,2023-08-26,type 4,act2_1734928
1,0,type 1,ppl_100,2022-09-27,type 2,act2_2434093
2,0,type 1,ppl_100,2022-09-27,type 2,act2_3404049
3,0,type 1,ppl_100,2023-08-04,type 2,act2_3651215
4,0,type 1,ppl_100,2023-08-26,type 2,act2_4109017


In [230]:
#Rename the 2 columns to avoid name clashes in merged data
df = df.rename(columns={"date":"activity_date","char_10":"activity_type"})

In [231]:
df.head()

,outcome,activity_type,people_id,activity_date,activity_category,activity_id
0,0,type 76,ppl_100,2023-08-26,type 4,act2_1734928
1,0,type 1,ppl_100,2022-09-27,type 2,act2_2434093
2,0,type 1,ppl_100,2022-09-27,type 2,act2_3404049
3,0,type 1,ppl_100,2023-08-04,type 2,act2_3651215
4,0,type 1,ppl_100,2023-08-26,type 2,act2_4109017


In [232]:
#The mode of a set of data values is the value that appears most often.
df["activity_type"].mode()

0    type 1
dtype: object

In [233]:
#Replace nulls in the activity_type column with the mode
df["activity_type"] = df["activity_type"].fillna(df["activity_type"].mode()[0])

In [234]:
#Print the shape of the final activity dataset
print("Old Shape:","(2197291, 15)")
print("New Shape:",df.shape)

Old Shape: (2197291, 15)
New Shape: (2197291, 6)


In [235]:
#Merge the 2 datasets on 'people_id' key
df_new = df.merge(people,on=["people_id"],how="inner")

In [236]:
df_new.head()

,outcome,activity_type,people_id,activity_date,activity_category,activity_id,char_1,group_1,char_2,date,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,0,type 76,ppl_100,2023-08-26,type 4,act2_1734928,type 2,group 17304,type 2,2021-06-29,...,False,True,True,False,False,True,True,True,False,36
1,0,type 1,ppl_100,2022-09-27,type 2,act2_2434093,type 2,group 17304,type 2,2021-06-29,...,False,True,True,False,False,True,True,True,False,36
2,0,type 1,ppl_100,2022-09-27,type 2,act2_3404049,type 2,group 17304,type 2,2021-06-29,...,False,True,True,False,False,True,True,True,False,36
3,0,type 1,ppl_100,2023-08-04,type 2,act2_3651215,type 2,group 17304,type 2,2021-06-29,...,False,True,True,False,False,True,True,True,False,36
4,0,type 1,ppl_100,2023-08-26,type 2,act2_4109017,type 2,group 17304,type 2,2021-06-29,...,False,True,True,False,False,True,True,True,False,36


In [237]:
print("Shape before merging:",df.shape)
print("Shape after merging :",df_new.shape)

Shape before merging: (2197291, 6)
Shape after merging : (2197291, 46)


##### The consistent number of rows and the increase in the number of columns helps us validate that the join operation worked as expected.

Let us now study the target (i.e., the variable we want to predict), named “outcome” in the dataset. 
We can check the distribution between potential vs. nonpotential customers.

In [238]:
print("Unique values for outcome:",df_new["outcome"].unique())
print("\nPercentage of distribution for outcome-")
print(df_new["outcome"].value_counts()/df_new.shape[0])

Unique values for outcome: [0 1]

Percentage of distribution for outcome-
0    0.556046
1    0.443954
Name: outcome, dtype: float64


##### We can see that there is a good mix in the distribution of potentialcustomers, as around 45% are potential customers.

## 3) Data Engineering

In [239]:
#Checking the distinct datatypes in the dataset
print("Distinct DataTypes:",list(df_new.dtypes.unique()))

Distinct DataTypes: [dtype('int64'), dtype('O'), dtype('bool')]


* We have numeric, categorical (Object), and Boolean features in the dataset. 
* Boolean in Python represents a True or False value; 
* We need to convert this into numeric (1 and 0) for the model to process the data.

In [240]:
#Create a temp dataset with the datatype of columns
temp = pd.DataFrame(df_new.dtypes)
temp.columns = ["DataType"]

In [241]:
temp

,DataType
outcome,int64
activity_type,object
people_id,object
activity_date,object
activity_category,object
activity_id,object
char_1,object
group_1,object
char_2,object
date,object


In [242]:
#Create a list with names of all Boolean columns
boolean_columns = temp.index[temp["DataType"] == 'bool'].values
print("Boolean columns - \n",boolean_columns)

Boolean columns - 
 ['char_10' 'char_11' 'char_12' 'char_13' 'char_14' 'char_15' 'char_16'
 'char_17' 'char_18' 'char_19' 'char_20' 'char_21' 'char_22' 'char_23'
 'char_24' 'char_25' 'char_26' 'char_27' 'char_28' 'char_29' 'char_30'
 'char_31' 'char_32' 'char_33' 'char_34' 'char_35' 'char_36' 'char_37']


In [243]:
#Convert all boolean columns to Binary numeric values
for column in boolean_columns:
    df_new[column] = np.where(df_new[column] == True,1,0)

print("\nDistinct DataTypes after processing:",df.dtypes.unique())


Distinct DataTypes after processing: [dtype('int64') dtype('O')]


In [244]:
#Extracting the object columns from the above dataframe
categorical_columns = temp.index[temp["DataType"] == 'O'].values

In [245]:
categorical_columns

array(['activity_type', 'people_id', 'activity_date', 'activity_category',
       'activity_id', 'char_1', 'group_1', 'char_2', 'date', 'char_3',
       'char_4', 'char_5', 'char_6', 'char_7', 'char_8', 'char_9'],
      dtype=object)

In [246]:
#Check the number of distinct values in each categorical column
for column in categorical_columns:
    print(column+" column has :",str(len(df_new[column].unique()))+" distinct values")

activity_type column has : 6515 distinct values
people_id column has : 151295 distinct values
activity_date column has : 411 distinct values
activity_category column has : 7 distinct values
activity_id column has : 2197291 distinct values
char_1 column has : 2 distinct values
group_1 column has : 29899 distinct values
char_2 column has : 3 distinct values
date column has : 1196 distinct values
char_3 column has : 43 distinct values
char_4 column has : 25 distinct values
char_5 column has : 9 distinct values
char_6 column has : 7 distinct values
char_7 column has : 25 distinct values
char_8 column has : 8 distinct values
char_9 column has : 9 distinct values


The five highlighted columns as shown in the output have high numbers of distinct values. It would be difficult to convert them into a one-hot encoded form, as they will consume too much memory during processing. In case you have the luxury of surplus RAM, feel free to convert them to a one-hot encoded data form.

In [247]:
#Create date related features for 'date' in customer data
df_new["date"] = pd.to_datetime(df_new["date"])

df_new["Year"] = df_new["date"].dt.year
df_new["Month"] = df_new["date"].dt.month
df_new["Quarter"] = df_new["date"].dt.quarter
df_new["Week"] = df_new["date"].dt.week
df_new["WeekDay"] = df_new["date"].dt.weekday
df_new["Day"] = df_new["date"].dt.day

#Create date related features for 'date' in activity data
df_new["activity_date"] = pd.to_datetime(df_new["activity_date"])

df_new["Activity_Year"] = df_new["activity_date"].dt.year
df_new["Activity_Month"] = df_new["activity_date"].dt.month
df_new["Activity_Quarter"] = df_new["activity_date"].dt.quarter
df_new["Activity_Week"] = df_new["activity_date"].dt.week
df_new["Activity_WeekDay"] = df_new["activity_date"].dt.weekday
df_new["Activity_Day"] = df_new["activity_date"].dt.day

#Delete the original date columns
del(df_new["date"])
del(df_new["activity_date"])

print("Shape of data after create Date Features:",df_new.shape)

Shape of data after create Date Features: (2197291, 56)


In [248]:
print(df_new[["people_id","activity_type","activity_id","group_1"]].head())

  people_id activity_type   activity_id      group_1
0   ppl_100       type 76  act2_1734928  group 17304
1   ppl_100        type 1  act2_2434093  group 17304
2   ppl_100        type 1  act2_3404049  group 17304
3   ppl_100        type 1  act2_3651215  group 17304
4   ppl_100        type 1  act2_4109017  group 17304


* It seems that we can convert all of the preceding categorical columns into numeric by extracting the relevant numeric ID from each of them.
* Since each of these columns has values in the form of someText_ someNumber. 
* Rather than converting these categorical columns into a bloated one-hot encoded dataset, we can temporarily use them as numeric features. 
* However, if the performance of the model doesn’t reach our desired expectations after several experiments, we might have to revisit these features and try our best to incorporate them differently. But for now, we can consider them as numeric features.

In [249]:
#For people ID, we would need to extract values after '_'
df_new.people_id = df_new.people_id.apply(lambda x:x.split("_")[1])
df_new.people_id = pd.to_numeric(df_new.people_id)

#For activity ID also, we would need to extract values after '_'
df_new.activity_id = df_new.activity_id.apply(lambda x:x.split("_")[1])
df_new.activity_id = pd.to_numeric(df_new.activity_id)

#For group_1 , we would need to extract values after ' '
df_new.group_1 = df_new.group_1.apply(lambda x:x.split(" ")[1])
df_new.group_1 = pd.to_numeric(df_new.group_1)

#For activity_type , we would need to extract values after ' '
df_new.activity_type = df_new.activity_type.apply(lambda x:x.split(" ")[1])
df_new.activity_type = pd.to_numeric(df_new.activity_type)

#Double check the new values in the dataframe
print(df_new[["people_id","activity_type","activity_id","group_1"]].head())

   people_id  activity_type  activity_id  group_1
0      100.0             76    1734928.0    17304
1      100.0              1    2434093.0    17304
2      100.0              1    3404049.0    17304
3      100.0              1    3651215.0    17304
4      100.0              1    4109017.0    17304


Next, let’s convert the remaining categorical columns, which have relatively low numbers of distinct values, to one-hot encoded form and render the final consolidated dataset.

In [250]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

#Creating a temporary dataframe that hosts column datatypes
temp = pd.DataFrame(df_new.dtypes)
temp.columns = ["DataType"]

In [251]:
temp

,DataType
outcome,int64
activity_type,int64
people_id,float64
activity_category,object
activity_id,float64
char_1,object
group_1,int64
char_2,object
char_3,object
char_4,object


In [252]:
#Create lists to separate numeric, categorical and target features
target = ["outcome"]
numeric_columns = list(set(temp.index[(temp.DataType =="float64") | (temp.DataType =="int64")].values) - set(target))
categorical_columns = temp.index[temp.DataType == "O"].values

In [253]:
categorical_columns

array(['activity_category', 'char_1', 'char_2', 'char_3', 'char_4',
       'char_5', 'char_6', 'char_7', 'char_8', 'char_9'], dtype=object)

In [254]:
print("\nTarget column :\n",target)
print("\nNumeric columns :\n",numeric_columns)
print("\nCategorical columns :\n",categorical_columns)


Target column :
 ['outcome']

Numeric columns :
 ['Month', 'activity_type', 'Activity_WeekDay', 'Activity_Week', 'char_38', 'Activity_Day', 'Quarter', 'Day', 'people_id', 'Week', 'WeekDay', 'group_1', 'Activity_Quarter', 'Activity_Year', 'activity_id', 'Activity_Month', 'Year']

Categorical columns :
 ['activity_category' 'char_1' 'char_2' 'char_3' 'char_4' 'char_5' 'char_6'
 'char_7' 'char_8' 'char_9']


In [255]:
#Define a function that will intake the raw dataframe and the column name and return a one hot encoded DF
def create_ohe(df, col):
    le = LabelEncoder()
    a=le.fit_transform(df_new[col]).reshape(-1,1)
    ohe = OneHotEncoder(sparse=False)
    column_names = [col+ "_"+ str(i) for i in le.classes_]
    return(pd.DataFrame(ohe.fit_transform(a),columns =column_names))

In [256]:
#Since the above function coverts the column, one at a time
#We create a loop to create the final dataset with all features
temp = df_new[numeric_columns]
for column in categorical_columns:
    temp_df = create_ohe(df_new,column)
    temp = pd.concat([temp,temp_df],axis=1)

print("\nShape of final df after onehot encoding:",temp.shape)

C:\Users\Gulraiz\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\Gulraiz\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case yo


Shape of final df after onehot encoding: (2197291, 155)


In [258]:
from sklearn.model_selection import train_test_split

#split the final dataset into train and test with 80:20
x_train, x_test, y_train, y_test = train_test_split(temp,df_new[target], test_size=0.2,random_state=2018)

#split the train dataset further into train and validation with 90:10
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train,test_size=0.1,random_state=2018)

#Check the shape of each new dataset created
print("Shape of x_train:",x_train.shape)
print("Shape of x_test:",x_test.shape)
print("Shape of x_val:",x_val.shape)
print("Shape of y_train:",y_train.shape)
print("Shape of y_test:",y_test.shape)
print("Shape of y_val:",y_val.shape)


Shape of x_train: (1582048, 155)
Shape of x_test: (439459, 155)
Shape of x_val: (175784, 155)
Shape of y_train: (1582048, 1)
Shape of y_test: (439459, 1)
Shape of y_val: (175784, 1)


## Designing a Deep Neural Network

In [259]:
from keras import Sequential
from keras.layers import Dense

#Design the deep neural network [Small + 1 layer]
model  = Sequential()
model.add(Dense(256,input_dim = x_train.shape[1],activation="relu"))
model.add(Dense(256,activation="relu"))
model.add(Dense(1,activation = "sigmoid")) #activation = sigmoid for binary classification

model.compile(optimizer = "Adam",loss="binary_crossentropy",metrics=["accuracy"])

model.fit(x_train,y_train, validation_data = (x_val,y_val),epochs=3, batch_size=64)


Using TensorFlow backend.







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 1582048 samples, validate on 175784 samples
Epoch 1/3
1582048/1582048 [==============================] - 135s 85us/step - loss: 7.1534 - acc: 0.5562 - val_loss: 7.1813 - val_acc: 0.5545
Epoch 2/3
1582048/1582048 [==============================] - 134s 85us/step - loss: 7.1534 - acc: 0.5562 - val_loss: 7.1813 - val_acc: 0.5545
Epoch 3/3
1582048/1582048 [==============================] - 133s 84us/step - loss: 7.1534 - acc: 0.5562 - val_loss: 7.1813 - val_acc: 0.5545


In [260]:
#Design the deep neural network [Small + 2 layers]
model  = Sequential()
model.add(Dense(256,input_dim = x_train.shape[1],activation="relu"))
model.add(Dense(256,activation="relu"))
model.add(Dense(1,activation = "sigmoid"))

model.compile(optimizer = "Adam",loss="binary_crossentropy",metrics=["accuracy"])

model.fit(x_train,y_train, validation_data = (x_val,y_val),epochs=3, batch_size=64)


Train on 1582048 samples, validate on 175784 samples
Epoch 1/3
1582048/1582048 [==============================] - 124s 78us/step - loss: 7.1534 - acc: 0.5562 - val_loss: 7.1813 - val_acc: 0.5545
Epoch 2/3
1582048/1582048 [==============================] - 125s 79us/step - loss: 7.1534 - acc: 0.5562 - val_loss: 7.1813 - val_acc: 0.5545
Epoch 3/3
1582048/1582048 [==============================] - 127s 80us/step - loss: 7.1534 - acc: 0.5562 - val_loss: 7.1813 - val_acc: 0.5545


In [261]:
#Design the deep neural network [Medium + 1 layers]
model  = Sequential()
model.add(Dense(512,input_dim = x_train.shape[1],activation="relu"))
model.add(Dense(1,activation = "sigmoid"))

model.compile(optimizer = "Adam",loss="binary_crossentropy",metrics=["accuracy"])

model.fit(x_train,y_train, validation_data = (x_val,y_val),epochs=3, batch_size=64)

Train on 1582048 samples, validate on 175784 samples
Epoch 1/3
1582048/1582048 [==============================] - 113s 71us/step - loss: 8.3515 - acc: 0.4776 - val_loss: 8.8394 - val_acc: 0.4455
Epoch 2/3
1582048/1582048 [==============================] - 117s 74us/step - loss: 8.8669 - acc: 0.4438 - val_loss: 8.8394 - val_acc: 0.4455
Epoch 3/3
1582048/1582048 [==============================] - 114s 72us/step - loss: 8.8669 - acc: 0.4438 - val_loss: 8.8394 - val_acc: 0.4455


In [262]:
#Design the deep neural network [Medium + 2 layers]
model  = Sequential()
model.add(Dense(512,input_dim = x_train.shape[1],activation="relu"))
model.add(Dense(512,activation="relu"))
model.add(Dense(1,activation = "sigmoid"))

model.compile(optimizer = "Adam",loss="binary_crossentropy",metrics=["accuracy"])

model.fit(x_train,y_train, validation_data = (x_val,y_val),epochs=3, batch_size=64)

Train on 1582048 samples, validate on 175784 samples
Epoch 1/3
1582048/1582048 [==============================] - 209s 132us/step - loss: 8.8426 - acc: 0.4454 - val_loss: 8.8394 - val_acc: 0.4455
Epoch 2/3
1582048/1582048 [==============================] - 209s 132us/step - loss: 8.8669 - acc: 0.4438 - val_loss: 8.8394 - val_acc: 0.4455
Epoch 3/3
1582048/1582048 [==============================] - 207s 131us/step - loss: 8.8669 - acc: 0.4438 - val_loss: 8.8394 - val_acc: 0.4455


## Revisiting the training data

In [263]:
#Create a dummy input
dummy_input = np.arange(1,10)
print("Dummy Input = ",dummy_input)

from sklearn.preprocessing  import StandardScaler

#Create a standardscaler instance and fit the data
scaler = StandardScaler()
output = scaler.fit_transform(dummy_input.reshape(-1,1))

print("Output =\n ",list(output))
print("Output's Mean = ",output.mean())
print("Output's Std Dev = ",output.std())
print("\nAfter Inverse Transforming = \n",list(scaler.inverse_transform(output)))

Dummy Input =  [1 2 3 4 5 6 7 8 9]
Output =
  [array([-1.54919334]), array([-1.161895]), array([-0.77459667]), array([-0.38729833]), array([0.]), array([0.38729833]), array([0.77459667]), array([1.161895]), array([1.54919334])]
Output's Mean =  0.0
Output's Std Dev =  1.0

After Inverse Transforming = 
 [array([1.]), array([2.]), array([3.]), array([4.]), array([5.]), array([6.]), array([7.]), array([8.]), array([9.])]


C:\Users\Gulraiz\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int32 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\Gulraiz\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int32 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [264]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x_train)

x_train_scaled = scaler.transform(x_train)
x_val_scaled = scaler.transform(x_val)
x_test_scaled = scaler.transform(x_test)


C:\Users\Gulraiz\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Gulraiz\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  
C:\Users\Gulraiz\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  import sys
C:\Users\Gulraiz\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


## Deep Neural Networks for Classification with improved data
Let us now, start with a medium sized network to see if we get improved results. We would start with just 3 epochs.

In [265]:
from keras import Sequential
from keras.layers import Dense
model  = Sequential()
model.add(Dense(512,input_dim = x_train_scaled.shape[1],activation="relu"))
model.add(Dense(1,activation = "sigmoid"))

model.compile(optimizer = "Adam",loss="binary_crossentropy",metrics=["accuracy"])

model.fit(x_train_scaled,y_train, 
validation_data = (x_val_scaled,y_val),epochs=3, batch_size=64)

Train on 1582048 samples, validate on 175784 samples
Epoch 1/3
1582048/1582048 [==============================] - 114s 72us/step - loss: 0.2521 - acc: 0.8891 - val_loss: 0.2179 - val_acc: 0.9093
Epoch 2/3
1582048/1582048 [==============================] - 116s 73us/step - loss: 0.2020 - acc: 0.9179 - val_loss: 0.1905 - val_acc: 0.9253
Epoch 3/3
1582048/1582048 [==============================] - 114s 72us/step - loss: 0.1782 - acc: 0.9308 - val_loss: 0.1752 - val_acc: 0.9333


In [266]:
result = model.evaluate(x_test_scaled,y_test)
for i in range(len(model.metrics_names)):
    print("Metric ",model.metrics_names[i],":",str(round(result[i],2)))

439459/439459 [==============================] - 14s 32us/step
Metric  loss : 0.18
Metric  acc : 0.93


In [267]:
#Designing the Deep Neural Network [Medium – 2 Layers]
model  = Sequential()
model.add(Dense(512,input_dim = x_train_scaled.shape[1],activation="relu"))
model.add(Dense(512,activation="relu"))
model.add(Dense(1,activation = "sigmoid"))

model.compile(optimizer = "Adam",loss="binary_crossentropy",metrics=["accuracy"])

model.fit(x_train_scaled,y_train, validation_data = (x_val_scaled,y_val),epochs=3, batch_size=64)

Train on 1582048 samples, validate on 175784 samples
Epoch 1/3
1582048/1582048 [==============================] - 220s 139us/step - loss: 0.2168 - acc: 0.9050 - val_loss: 0.1620 - val_acc: 0.9318
Epoch 2/3
1582048/1582048 [==============================] - 296s 187us/step - loss: 0.1405 - acc: 0.9431 - val_loss: 0.1283 - val_acc: 0.9486
Epoch 3/3
1582048/1582048 [==============================] - 286s 181us/step - loss: 0.1116 - acc: 0.9559 - val_loss: 0.1090 - val_acc: 0.9579


In [ ]:
#Designing the network Deep Neural Network – [Large + 2 Layers]
model  = Sequential()
model.add(Dense(1024,input_dim = x_train_scaled.shape[1],activation="relu"))
model.add(Dense(1024,activation = "relu"))
model.add(Dense(1,activation = "sigmoid"))

model.compile(optimizer = "Adam",loss="binary_crossentropy",metrics=["accuracy"])

model.fit(x_train_scaled,y_train, 
validation_data = (x_val_scaled,y_val),epochs=3, batch_size=64)


In [ ]:
#Designing the network Deep Neural Network – [Medium + 2 Layers]
model  = Sequential()
model.add(Dense(512,input_dim = x_train_scaled.shape[1],activation="relu"))
model.add(Dense(512,activation = "relu"))
model.add(Dense(1,activation = "sigmoid"))

model.compile(optimizer = "Adam",loss="binary_crossentropy",metrics=["accuracy"])

model.fit(x_train_scaled,y_train, validation_data = (x_val_scaled,y_val),epochs=15, batch_size=64)

In [ ]:
result = model.evaluate(x_test_scaled,y_test)

for i in range(len(model.metrics_names)):
    print("Metric ",model.metrics_names[i],":",str(round(result[i],2)))

## Plotting the final results

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(model.history.history['loss'])
plt.plot(model.history.history['val_loss'])
plt.title("Model's Training & Validation loss across epochs")
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

In [ ]:
plt.plot(model.history.history['acc'])
plt.plot(model.history.history['val_acc'])
plt.title("Model's Training & Validation Accuracy across epochs")
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()